<a href="https://colab.research.google.com/github/Sai-Shashank/AI-Genetic-Algorithm/blob/main/Open_Face_Feature_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prepare the training and testing dataset

In [ ]:
from google.colab import files
uploaded= files.upload()


Saving final-labels.csv to final-labels.csv
Saving reps-final.csv to reps-final.csv


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

rep_data = pd.read_csv('reps-final.csv', sep=',', header=0)
label_data = pd.read_csv('final-labels.csv', sep=',', header=0)



df = pd.DataFrame(rep_data.to_numpy(), columns=[i for i in range(0,len(rep_data.to_numpy()[0]))])

le = LabelEncoder()
label = le.fit_transform(label_data)


X_train, X_test, y_train, y_test = train_test_split(df, 
                                                    label, test_size=0.20, 
                                                    random_state=101)



/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:251: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


# Train using all the features (Random Forest Classifier)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

classifier = RandomForestClassifier(n_estimators=20,random_state=101)
classifier.fit(X_train,y_train.ravel())
y_pred = classifier.predict(X_test)

from sklearn.metrics import accuracy_score
print(accuracy_score(y_test.ravel(),y_pred))

0.9521276595744681


# Defining various Genetic Algorithm Operators

Selection Operators :


In [ ]:
def roulette_selection(pop_after_fit,n_parents,scores):
    total_fitness_sum = np.sum(scores)
    probability_of_selection = []
    population_after_selection = []
    length = len(scores)
    for i in range(0,length):
      probability = scores[i]/total_fitness_sum
      probability_of_selection.append(probability)
    pop_indices=[]
    for i in range(0,length):
      pop_indices.append(i)
    from numpy.random import choice
    pop_indices = choice(pop_indices,length,probability_of_selection)
    for i in range(0,length):
      population_after_selection.append(pop_after_fit[pop_indices[i]])
    return population_after_selection  

def tournament_selection(pop_after_fit, n_parents,scores):
    pop_after_selection = list()
    pop_after_fit_numpy = np.array(pop_after_fit)
    scores_numpy = np.array(scores)
    for i in range(int(n_parents)):
      rand_ints = random.sample(range(0,n_parents),16)
      init_pop = pop_after_fit_numpy[rand_ints]
      init_score = scores_numpy[rand_ints]
      max_score_ind = np.argmax(init_score)
      pop_after_selection.append(pop_after_fit_numpy[max_score_ind])
            
    return pop_after_selection


def rank_based_selection(pop_after_fit, n_parents):
    probability_of_selection = []
    length = len(pop_after_fit)
    population_after_selection = []
    dec = 1/length
    curr_val = 1
    for i in range(0, length):
      probability_of_selection.append(curr_val)
      curr_val -= dec
    pop_indices=[]
    for i in range(0,length):
      pop_indices.append(i)
    from numpy.random import choice
    pop_indices = choice(pop_indices,length,probability_of_selection)
    for i in range(0,length):
      population_after_selection.append(pop_after_fit[pop_indices[i]])
    return population_after_selection
    
    



Crossover Operators:

In [ ]:
def single_point_crossover(pop_after_sel):
  population_nextgen=pop_after_sel
  b = random.randint(1,710)
  for i in range(len(pop_after_sel)):
    parent1 = pop_after_sel[i]
    parent2 = pop_after_sel[(i+1)%len(pop_after_sel)]
    first_child_genes1 = parent1[0:b]
    first_child_genes2 = parent2[b:]      
    first_child = []
    for x in first_child_genes1:
      first_child.append(x)
    for x in first_child_genes2:
      first_child.append(x)
    population_nextgen.append(first_child)
  return population_nextgen

def double_point_crossover(pop_after_sel):
  population_nextgen=pop_after_sel
  for i in range(len(pop_after_sel)):
      child=pop_after_sel[i]
      child[50:100]=pop_after_sel[(i+1)%len(pop_after_sel)][50:100]
      population_nextgen.append(child)
  return population_nextgen

def uniform_crossover(pop_after_sel):
  pop_nextgen = pop_after_sel
  for i in range(0,len(pop_after_sel)):
    parent1 = pop_after_sel[i]
    parent2 = pop_after_sel[(i+1)%len(pop_after_sel)]
    child = []
    for j in range(0,len(parent1)):
      val = random.choice([parent1[j],parent2[j]])
      child.append(val)
    pop_nextgen.append(child)
  return pop_nextgen


Mutation Operators

In [ ]:
def mutation_flip_bits(pop_after_cross,mutation_rate=0.05):
  population_nextgen = []
  for i in range(0,len(pop_after_cross)):
    chromosome = pop_after_cross[i]
    to_be_flipped = np.random.randint(low=0, high=len(pop_after_cross[i]), size=1)[0]
    if np.random.random() < mutation_rate:
      chromosome[to_be_flipped] = not chromosome[to_be_flipped]
    # done = 1
    # for j in range(len(chromosome)):
    #   if random.random() < mutation_rate and done:
    #     chromosome[j]= not chromosome[j]
    #     done=0
    population_nextgen.append(chromosome)
  # print(population_nextgen)
  return population_nextgen

# generation = [[1, 0, 0, 1], [0, 1, 0, 1], [1, 0, 1, 0, 0, 1, 0]]
# mutation_flip_bits(generation)


def mutation_bit_swap(offspring_crossover, mutation_rate=0.1):
  for idx in range(0,len(offspring_crossover)):
    mutation_gene1 = np.random.randint(low=0, high=len(offspring_crossover[idx])/2)
    mutation_gene2 = mutation_gene1 + int(len(offspring_crossover[idx])/2)
    if np.random.random() < mutation_rate:
      temp = offspring_crossover[idx][mutation_gene1]
      offspring_crossover[idx][mutation_gene1] = offspring_crossover[idx][mutation_gene2]
      offspring_crossover[idx][mutation_gene2] = temp
  # print(offspring_crossover)
  return offspring_crossover

# generation = [[1, 1, 0, 0], [1, 1, 1, 1, 0, 0, 0], [1, 1, 1, 1, 0, 0, 0, 0]]
# mutation_bit_swap(generation)
  

Genetic Algorithm :

In [ ]:
#defining various steps required for the genetic algorithm
def initilization_of_population(size,n_feat):
    population = []
    for i in range(size):
        chromosome = np.ones(n_feat,dtype=np.bool)
        chromosome[:int(0.3*n_feat)]=False
        np.random.shuffle(chromosome)
        population.append(chromosome)
    return population

def fitness_score(population):
    scores = []
    for chromosome in population:
        classifier1 = RandomForestClassifier(n_estimators=20,random_state=0)
        classifier1.fit(X_train.iloc[:,chromosome],y_train.ravel())
        predictions = classifier1.predict(X_test.iloc[:,chromosome])
        scores.append(accuracy_score(y_test.ravel(),predictions)/sum(chromosome))
    scores, population = np.array(scores), np.array(population) 
    inds = np.argsort(scores)
    return list(scores[inds][::-1]), list(population[inds,:][::-1])

def selection(pop_after_fit,n_parents):
    population_nextgen = []
    for i in range(n_parents):
        population_nextgen.append(pop_after_fit[i])
    return population_nextgen

def crossover(pop_after_sel):
    population_nextgen=pop_after_sel
    for i in range(len(pop_after_sel)):
        child=pop_after_sel[i]
        child[3:7]=pop_after_sel[(i+1)%len(pop_after_sel)][3:7]
        population_nextgen.append(child)
    return population_nextgen

def mutation(pop_after_cross,mutation_rate):
    population_nextgen = []
    for i in range(0,len(pop_after_cross)):
        chromosome = pop_after_cross[i]
        for j in range(len(chromosome)):
            if random.random() < mutation_rate:
                chromosome[j]= not chromosome[j]
        population_nextgen.append(chromosome)
    #print(population_nextgen)
    return population_nextgen

def generations(size,n_feat,n_parents,mutation_rate,n_gen,X_train,
                                   X_test, y_train, y_test):
    best_chromo= []
    best_score= []
    population_nextgen=initilization_of_population(size,n_feat)
    for i in range(n_gen):
        scores, pop_after_fit = fitness_score(population_nextgen)
        print("Generation " + str(i) + ': ' + str(sum(scores[:10])/10))
        pop_after_sel = tournament_selection(pop_after_fit,n_parents,scores)
        pop_after_cross = single_point_crossover(pop_after_sel)
        population_nextgen = mutation_bit_swap(pop_after_cross,mutation_rate)
        best_chromo.append(pop_after_fit[0])
        best_score.append(scores[0])
    return best_chromo,best_score

# Applying Genetic Algorithms for Feature Selection

In [ ]:
import time
start = time.time()
import numpy as np
import random
 
chromo,score=generations(size=100,n_feat=711,n_parents=100,mutation_rate=0.10,
                     n_gen=5,X_train=X_train,X_test=X_test,y_train=y_train,y_test=y_test)

print('Done')
#print('Selected Features : ' + str(chromo[-1]))
classifier.fit(X_train.iloc[:,chromo[-1]],y_train)
predictions = classifier.predict(X_test.iloc[:,chromo[-1]])
print("Accuracy score after genetic algorithm is= "+str(accuracy_score(y_test,predictions)))
end = time.time()
time_taken = end-start
print('Time taken : '+str(time_taken))

Generation 0: 0.0019354011791848243
Generation 1: 0.0019580965704149517
Generation 2: 0.0019852315177207045
Generation 3: 0.0019998485957990917
Generation 4: 0.002020310363768193
Done
Accuracy score after genetic algorithm is= 0.9627659574468085
Time taken : 293.2268011569977


Displaying the optimal solution

In [ ]:
df = pd.read_csv('reps-final.csv')
header_list = df.columns
sol = chromo[-1]
ans = []
for i in range(0,len(sol)):
  if sol[i] == True:
    ans.append(header_list[i])
print(ans)
print(len(ans))


[' confidence', ' gaze_0_y', ' gaze_0_z', ' gaze_1_x', ' gaze_1_y', ' gaze_angle_x', ' gaze_angle_y', ' eye_lmk_x_0', ' eye_lmk_x_1', ' eye_lmk_x_2', ' eye_lmk_x_4', ' eye_lmk_x_5', ' eye_lmk_x_6', ' eye_lmk_x_9', ' eye_lmk_x_11', ' eye_lmk_x_12', ' eye_lmk_x_13', ' eye_lmk_x_14', ' eye_lmk_x_15', ' eye_lmk_x_16', ' eye_lmk_x_19', ' eye_lmk_x_20', ' eye_lmk_x_21', ' eye_lmk_x_22', ' eye_lmk_x_23', ' eye_lmk_x_24', ' eye_lmk_x_28', ' eye_lmk_x_29', ' eye_lmk_x_30', ' eye_lmk_x_31', ' eye_lmk_x_32', ' eye_lmk_x_34', ' eye_lmk_x_36', ' eye_lmk_x_37', ' eye_lmk_x_38', ' eye_lmk_x_39', ' eye_lmk_x_40', ' eye_lmk_x_41', ' eye_lmk_x_44', ' eye_lmk_x_45', ' eye_lmk_x_46', ' eye_lmk_x_47', ' eye_lmk_x_49', ' eye_lmk_x_51', ' eye_lmk_x_52', ' eye_lmk_x_53', ' eye_lmk_x_54', ' eye_lmk_y_0', ' eye_lmk_y_1', ' eye_lmk_y_4', ' eye_lmk_y_5', ' eye_lmk_y_6', ' eye_lmk_y_7', ' eye_lmk_y_8', ' eye_lmk_y_11', ' eye_lmk_y_12', ' eye_lmk_y_13', ' eye_lmk_y_14', ' eye_lmk_y_15', ' eye_lmk_y_16', ' eye_lmk_y

In [ ]:
arr = [True,False]
print(sum(arr))

1
